In [8]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
import numpy as np

# grover의 oracle - target |11>을 찾는다고 가정
def create_oracle(circuit, qubits, target ='11'):
    """
    oracle: 찾고자 하는 상태에 -1 위상을 붙임
    target='11'이면 |11> 상태만 표시
    """
    if target == '11':
        # |11>만 표시 : z 게이트를 양쪽에
        circuit.cz(qubits[0], qubits[1])
    elif target == '10':
        # |10>을 표시
        circuit.x(qubits[1])
        circuit.cz(qubits[0], qubits[1])
        circuit.x(qubits[1])
    elif target == '01':
        # |01>을 표시
        circuit.x(qubits[0])
        circuit.cz(qubits[0], qubits[1])
        circuit.x(qubits[0])
    elif target == '00':
        # |00>을 표시
        circuit.x(qubits[0])
        circuit.x(qubits[1])
        circuit.cz(qubits[0], qubits[1])
        circuit.x(qubits[0])
        circuit.x(qubits[1])

# grover의 diffusion operator (증폭)
def create_diffuser(circuit, qubits):
    """
    Diffusion : 표시된 상태의 진폭을 증폭
    2*|평균><평균| - ㅑ
    """
    n= len(qubits)

    # h 게이트
    for qubit in qubits:
        circuit.h(qubit)

    # x 게이트
    for qubit in qubits:
        circuit.x(qubit)

    # multi-controlled z
    circuit.h(qubits[-1])
    if n ==2:
        circuit.cx(qubits[0], qubits[1])
    circuit.h(qubits[-1])

    # x 게이트 (되돌림)
    for qubit in qubits:
        circuit.x(qubit)

    # h 게이트
    for qubit in qubits:
        circuit.h(qubit)

# grover 회로 생성
def grover_circuit(target='11', iterations=1):
    """
    2큐비트 grover 알고리즘
    target : 찾고자 하는 상태 ('00', '01', '10', '11')
    iterations : grover 반복 횟수 (2큐비트는 1회가 최적)
    """
    # 레지스터 생성
    qubits = QuantumRegister(2, 'q')
    classical = ClassicalRegister(2, 'c')
    qc = QuantumCircuit(qubits, classical)

    # 1. 초기 상태 : 균등 중첩 생성
    qc.h(qubits[0])
    qc.h(qubits[1])
    qc.barrier(label='초기 중첩')

    # 2. grover 반복 (oracle + Diffusion)
    for i in range(iterations):
        # oracle:목표 상태 표시
        create_oracle(qc, qubits, target)
        qc.barrier(label=f'Oracle {i+1}')

        # Diffusion : 진폭 증폭
        create_diffuser(qc, qubits)
        qc.barrier(label=f'diffusion {i+1}')

    # 3. 측정
    qc.measure(qubits, classical)

    return qc

# 실행
service = QiskitRuntimeService(
    channel='ibm_quantum_platform',
    token = '')

backend = service.least_busy(simulator=False, operational=True, min_num_qubits=2)
print(f"백엔드 : {backend.name}")

# 찾고자 하는 목표 설정
TARGET = '10' # |11> 찾기

# 회로 생성 (1회 반복)
qc = grover_circuit(target=TARGET, iterations=1)

print(f"\n 회로 깊이 : {qc.depth()}")
print(f" 게이트 수: {qc.size()}")

# 트랜스파이 & 실행
pm = generate_preset_pass_manager(backend=backend, optimization_level=3)
isa_circuit = pm.run(qc)

print(f"\n 최적화 후 깊이 : {isa_circuit.depth()}")

sampler = Sampler(backend)
job = sampler.run([isa_circuit], shots = 1024)
print(f"\n Job ID: {job.job_id()}")
print(" 실행중...")

result = job.result()
counts_raw = result[0].data.c.get_counts()
counts = {k[::-1]: v for k, v in counts_raw.items()}

# 결과
print(f"\n{'='*50}")
print(f" 목표 : |{TARGET}>")
print(f"{'='*50}")

for bitstring, count in sorted(counts.items(), key=lambda x: x[1], reverse=True):
    percentage = count / 1024 * 100
    bar = '█' * int(percentage / 2)

    # 목표와 일치하면 강조
    if bitstring == TARGET :
        print(f" |{bitstring}>: {count:4d}회 ({percentage:5.1f}%) {bar} <- TARGET!")
    else:
        print(f" |{bitstring}>: {count:4d}회 ({percentage:5.1f}%) {bar}")

# 성공률 계산
if TARGET in counts:
    success_rate = counts[TARGET] / 1024 * 100
    print(f"\n 성공률 : {success_rate:.1f}%")

    # 이론적 활률 (2큐비트, 1회 반복)
    theoretical = 100.0 # 이론상 100%
    print(f" 이론값: {theoretical:.1f}%")
    print(f" 노이즈 영향: {theoretical - success_rate:.1f}%")
else:
    print(f"\n 목표 상태 발견 안 됨!")
        

qiskit_runtime_service.__init__:WARNING:2025-12-21 23:57:36,819: Instance was not set at service instantiation. Free and trial plan instances will be prioritized. Based on the following filters: (tags: None, region: us-east, eu-de), and available plans: (open), the available account instances are: dotnanimoe. If you need a specific instance set it explicitly either by using a saved account with a saved default instance or passing it in directly to QiskitRuntimeService().
qiskit_runtime_service.backends:WARNING:2025-12-21 23:57:37,444: Loading instance: dotnanimoe, plan: open
qiskit_runtime_service.backends:WARNING:2025-12-21 23:57:39,953: Using instance: dotnanimoe, plan: open


백엔드 : ibm_torino

 회로 깊이 : 12
 게이트 수: 18

 최적화 후 깊이 : 14

 Job ID: d540lp9smlfc739fahtg
 실행중...

 목표 : |10>
 |10>:  979회 ( 95.6%) ███████████████████████████████████████████████ <- TARGET!
 |00>:   19회 (  1.9%) 
 |01>:   17회 (  1.7%) 
 |11>:    9회 (  0.9%) 

 성공률 : 95.6%
 이론값: 100.0%
 노이즈 영향: 4.4%
